In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

#My imports
import wrangle as w
import env
import model as m

In [2]:
df = w.get_zillow_data()

In [3]:
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


In [4]:
#Clean the zillow data
df = w.clean_zillow(df)

In [5]:
#Remove outliers
df = w.remove_outliers(df,['bedrooms','bathrooms','square_feet','tax_value','tax_amount'])

In [6]:
#Prepare for modeling
df.head(2)

,bedrooms,bathrooms,square_feet,tax_value,year_built,tax_amount,fips,county,home_age
4,4,2.0,3633,296425,2005,6941.39,6037,Los Angeles,12
6,3,4.0,1620,847770,2011,10244.94,6037,Los Angeles,6


In [7]:
# Drop columns for modeling that are not needed (Year built, fips and tax amount (too similar to target variable))
df = df.drop(['year_built', 'fips', 'tax_amount'], axis=1)

In [8]:
df.head(2)

,bedrooms,bathrooms,square_feet,tax_value,county,home_age
4,4,2.0,3633,296425,Los Angeles,12
6,3,4.0,1620,847770,Los Angeles,6


In [9]:
#Make dummy variables from county
# Creating dummy variables:
df = pd.get_dummies(df, columns=['county'], drop_first=False)

In [10]:
df.head(2)

,bedrooms,bathrooms,square_feet,tax_value,home_age,county_Los Angeles,county_Orange,county_Ventura
4,4,2.0,3633,296425,12,1,0,0
6,3,4.0,1620,847770,6,1,0,0
